In [1]:
pip install pflacco

  Using cached pflacco-1.2.2-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached pyDOE-0.3.8-py3-none-any.whl
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached numdifftools-0.9.41-py2.py3-none-any.whl (100 kB)
  Using cached salib-1.4.7-py3-none-any.whl (757 kB)
  Using cached multiprocess-0.70.15-py310-none-any.whl.metadata (7.2 kB)
Using cached pflacco-1.2.2-py3-none-any.whl (41 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
Using cached multiprocess-0.70.15-py310-none-any.whl

In [2]:
pip install cocopp

  Using cached cocopp-2.6.3-py3-none-any.whl (4.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cocoex
from pflacco.sampling import create_initial_sample
from pflacco.classical_ela_features import *
import pandas as pd

In [2]:
BBOB2 = cocoex.Suite("bbob", f"instances:1-5", f"function_indices:1-24 dimensions:2,3,5,10")
print(BBOB2)

Suite("b'bbob'", "b'instances:1-5'", "b'function_indices:1-24 dimensions:2,3,5,10'") with 480 problems in dimensions 2=10


In [3]:

features = []

for problem in BBOB2:
    dim = problem.dimension
    fid = problem.id_function
    iid = problem.id_instance

    print(f"fid:{fid}, dim:{dim}, iid:{iid}")
    
    # Create sample
    X = create_initial_sample(dim, sample_type = 'lhs', lower_bound = -5, upper_bound = 5)
    y = X.apply(lambda x: problem(x), axis = 1)
   
    

    # Classical ELA features
    ela_meta = calculate_ela_meta(X, y)
    ela_distr = calculate_ela_distribution(X, y)
    ela_level = calculate_ela_level(X, y, ela_level_quantiles = [0.25,0.5], ela_level_resample_iterations = 5)

    # Compute the remaining 3 feature sets from the classical ELA features which do require additional function evaluations
    ela_local = calculate_ela_local(X, y, f = problem, dim = dim, lower_bound = -5, upper_bound = 5)
    ela_curv = calculate_ela_curvate(X, y, f = problem, dim = dim, lower_bound = -5, upper_bound = 5)
    ela_conv = calculate_ela_conv(X, y, f = problem)
    
    #Cell Mapping
    # Compute cell mapping angle feature set from the convential ELA features
    if(dim < 5):
        cm_angle = calculate_cm_angle(X, y, blocks = 3, lower_bound = -5, upper_bound = 5)
   
    
    
    #Dispersion features
    # Compute disp feature set from the convential ELA features
    disp = calculate_dispersion(X, y)
    
    
    #IC features
    # Compute ic feature set from the convential ELA features
    ic = calculate_information_content(X, y)
    
    #NBC features
    nbc = calculate_nbc(X, y)
    
    #Miscellaneous features
    #PCA features
    pca = calculate_pca(X,y)
    
    
    
    # cm_conv = calculate_cm_conv(X,y, lower_bound = -5, upper_bound = 5)
    # cm_grad = calculate_cm_grad(X,y, lower_bound = -5, upper_bound = 5)
    # limo = calculate_limo(X,y,lower_bound = -5, upper_bound = 5)
 
    data = pd.DataFrame({**{'fid': fid}, **{'dim': dim}, **{'iid': iid}, ** ela_meta,  **ela_distr, **ela_level, **ela_local, **ela_curv, **ela_conv, **disp, **ic, **nbc, **pca, **cm_angle}, index = [0])
    if(fid ==1 and iid == 1 and dim ==2):
        data.to_csv("data.csv",encoding='utf-8', index=False)
        print(data.shape)
    features.append(data)

#remove feature for dimensions with divide by zero error

ela_df = pd.concat(features).reset_index(drop = True)
print(f"Final row count of DataFrame: {ela_df.shape[0]}")
file_name = "features.csv"
ela_df.to_csv(file_name, encoding='utf-8', index=False)

fid:1, dim:2, iid:1
(1, 119)
fid:1, dim:2, iid:2
fid:1, dim:2, iid:3
fid:1, dim:2, iid:4
fid:1, dim:2, iid:5
fid:2, dim:2, iid:1
fid:2, dim:2, iid:2
fid:2, dim:2, iid:3
fid:2, dim:2, iid:4
fid:2, dim:2, iid:5
fid:3, dim:2, iid:1
fid:3, dim:2, iid:2
fid:3, dim:2, iid:3
fid:3, dim:2, iid:4


/opt/conda/lib/python3.10/site-packages/pflacco/classical_ela_features.py:1347: RuntimeWarning: invalid value encountered in scalar divide
  lda_qda = np.array([lda_mmce[i]/qda_mmce[i] for i in range(len(ela_level_quantiles))])


fid:3, dim:2, iid:5
fid:4, dim:2, iid:1
fid:4, dim:2, iid:2
fid:4, dim:2, iid:3
fid:4, dim:2, iid:4
fid:4, dim:2, iid:5
fid:5, dim:2, iid:1


/opt/conda/lib/python3.10/site-packages/pflacco/classical_ela_features.py:1588: RuntimeWarning: divide by zero encountered in scalar divide
  'ela_local.best2mean_contr.ratio': (center_fvals.mean() - center_fvals.min())/ (center_fvals.max() - center_fvals.min()),


fid:5, dim:2, iid:2


...

fid:5, dim:10, iid:4


/opt/conda/lib/python3.10/site-packages/pflacco/classical_ela_features.py:1588: RuntimeWarning: divide by zero encountered in scalar divide
  'ela_local.best2mean_contr.ratio': (center_fvals.mean() - center_fvals.min())/ (center_fvals.max() - center_fvals.min()),


fid:5, dim:10, iid:5


/opt/conda/lib/python3.10/site-packages/pflacco/classical_ela_features.py:1588: RuntimeWarning: invalid value encountered in scalar divide
  'ela_local.best2mean_contr.ratio': (center_fvals.mean() - center_fvals.min())/ (center_fvals.max() - center_fvals.min()),


fid:6, dim:10, iid:1
fid:6, dim:10, iid:2
fid:6, dim:10, iid:3
fid:6, dim:10, iid:4
fid:6, dim:10, iid:5
fid:7, dim:10, iid:1
fid:7, dim:10, iid:2
fid:7, dim:10, iid:3
fid:7, dim:10, iid:4
fid:7, dim:10, iid:5
fid:8, dim:10, iid:1
fid:8, dim:10, iid:2
fid:8, dim:10, iid:3
fid:8, dim:10, iid:4
fid:8, dim:10, iid:5
fid:9, dim:10, iid:1
fid:9, dim:10, iid:2
fid:9, dim:10, iid:3
fid:9, dim:10, iid:4
fid:9, dim:10, iid:5
fid:10, dim:10, iid:1
fid:10, dim:10, iid:2
fid:10, dim:10, iid:3
fid:10, dim:10, iid:4
fid:10, dim:10, iid:5
fid:11, dim:10, iid:1
fid:11, dim:10, iid:2
fid:11, dim:10, iid:3
fid:11, dim:10, iid:4
fid:11, dim:10, iid:5
fid:12, dim:10, iid:1
fid:12, dim:10, iid:2
fid:12, dim:10, iid:3
fid:12, dim:10, iid:4
fid:12, dim:10, iid:5
fid:13, dim:10, iid:1
fid:13, dim:10, iid:2
fid:13, dim:10, iid:3
fid:13, dim:10, iid:4
fid:13, dim:10, iid:5
fid:14, dim:10, iid:1
fid:14, dim:10, iid:2
fid:14, dim:10, iid:3
fid:14, dim:10, iid:4
fid:14, dim:10, iid:5
fid:15, dim:10, iid:1
fid:15, 

In [ ]:
print("Done")